In [ ]:
from criminologia_cdmx.etl import *
from criminologia_cdmx.patrones_espacio_temporales import *
from criminologia_cdmx.covariables import *
from criminologia_cdmx.modelos import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import h3
from shapely.geometry import Polygon
import contextily as ctx
import requests
from functools import reduce
from libpysal.weights import Queen, lag_spatial

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
manzanas = gpd.read_file("datos/manzanas_2020_cdmx.zip")
colonias = gpd.read_file("datos/criminologia_capas.gpkg", layer='colonias')
cuadrantes = gpd.read_file("datos/criminologia_capas.gpkg", layer='cuadrantes')
manzanas_centroides = manzanas.set_geometry(manzanas.geometry.centroid)
manzanas_centroides = (manzanas_centroides
                       .sjoin(colonias[['geometry', 'colonia_cve', 'colonia_nombre']]
                              .to_crs(manzanas.crs))
                       .drop(columns='index_right'))
manzanas_centroides = (manzanas_centroides
                       .sjoin(cuadrantes[['geometry', 'cuadrante_id']]
                              .to_crs(manzanas.crs))
                       .drop(columns='index_right'))
manzanas = (manzanas
            .merge(manzanas_centroides[['CVEGEO', 'colonia_cve', 
                                        'colonia_nombre', 'cuadrante_id']], on='CVEGEO'))
manzanas

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA,geometry,colonia_cve,colonia_nombre,cuadrante_id
0,0901000010898031,09,010,0001,0898,031,Urbana,Típica,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",1119,HERON PROAL,017
1,0901000012269024,09,010,0001,2269,024,Urbana,Típica,"POLYGON ((2791387.916 821465.156, 2791376.215 ...",1082,LOMAS DE TARANGO,014
2,0901000011472068,09,010,0001,1472,068,Urbana,Típica,"POLYGON ((2788494.355 822775.106, 2788478.422 ...",1030,GRAL A ROSALES,011
3,0901000011824024,09,010,0001,1824,024,Urbana,Típica,"POLYGON ((2790870.494 819456.298, 2790862.256 ...",1135,SAN CLEMENTE,0110
4,0901000012377004,09,010,0001,2377,004,Urbana,Típica,"POLYGON ((2792111.138 821977.886, 2792115.437 ...",1081,HOGAR Y REDENCION,018
...,...,...,...,...,...,...,...,...,...,...,...,...
66308,0900700015376020,09,007,0001,5376,020,Urbana,Edificio-Manzana,"POLYGON ((2805560.493 815680.504, 2805559.129 ...",1442,ZONA MILITAR FAVE SEDENA (U HAB),019
66309,0900700015376021,09,007,0001,5376,021,Urbana,Contenedora,"POLYGON ((2805372.709 815883.364, 2805488.537 ...",1442,ZONA MILITAR FAVE SEDENA (U HAB),019
66310,0900700013045056,09,007,0001,3045,056,Urbana,Edificio-Manzana,"POLYGON ((2809738.035 818480.137, 2809737.767 ...",1419,DESARROLLO URBANO QUETZALCOATL I,0113
66311,0900700013045032,09,007,0001,3045,032,Urbana,Contenedora,"POLYGON ((2809829.520 818566.731, 2809827.153 ...",1419,DESARROLLO URBANO QUETZALCOATL I,0113


In [ ]:
manzanas.to_file("datos/descargas/manzanas_identificadores.gpkg", layer='manzanas', driver="GPKG")

/home/plablo/miniconda3/envs/criminologia/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [ ]:
gpd.read_file("datos/descargas/manzanas_identificadores.gpkg")

,CVEGEO,CVE_ENT,CVE_MUN,CVE_LOC,CVE_AGEB,CVE_MZA,AMBITO,TIPOMZA,colonia_cve,colonia_nombre,cuadrante_id,geometry
0,0901000010898031,09,010,0001,0898,031,Urbana,Típica,1119,HERON PROAL,017,"POLYGON ((2790204.135 820779.247, 2790125.593 ..."
1,0901000012269024,09,010,0001,2269,024,Urbana,Típica,1082,LOMAS DE TARANGO,014,"POLYGON ((2791387.916 821465.156, 2791376.215 ..."
2,0901000011472068,09,010,0001,1472,068,Urbana,Típica,1030,GRAL A ROSALES,011,"POLYGON ((2788494.355 822775.106, 2788478.422 ..."
3,0901000011824024,09,010,0001,1824,024,Urbana,Típica,1135,SAN CLEMENTE,0110,"POLYGON ((2790870.494 819456.298, 2790862.256 ..."
4,0901000012377004,09,010,0001,2377,004,Urbana,Típica,1081,HOGAR Y REDENCION,018,"POLYGON ((2792111.138 821977.886, 2792115.437 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
66308,0900700015376020,09,007,0001,5376,020,Urbana,Edificio-Manzana,1442,ZONA MILITAR FAVE SEDENA (U HAB),019,"POLYGON ((2805560.493 815680.504, 2805559.129 ..."
66309,0900700015376021,09,007,0001,5376,021,Urbana,Contenedora,1442,ZONA MILITAR FAVE SEDENA (U HAB),019,"POLYGON ((2805372.709 815883.364, 2805488.537 ..."
66310,0900700013045056,09,007,0001,3045,056,Urbana,Edificio-Manzana,1419,DESARROLLO URBANO QUETZALCOATL I,0113,"POLYGON ((2809738.035 818480.137, 2809737.767 ..."
66311,0900700013045032,09,007,0001,3045,032,Urbana,Contenedora,1419,DESARROLLO URBANO QUETZALCOATL I,0113,"POLYGON ((2809829.520 818566.731, 2809827.153 ..."


In [ ]:
cuadrantes

,_id,id,no_regn,no_cdrn,zona,geo_shp,g_pnt_2,alcaldi,sector,clv_sct,cuadrante_id,geometry
0,1,0,1,8,PONIENTE,"{""type"": ""Polygon"", ""coordinates"": [[[-99.2138...","19.3279411421,-99.2101175866",ALVARO OBREGON,SAN ANGEL,3,018,"MULTIPOLYGON (((-99.21385 19.32244, -99.21397 ..."
1,2,1,1,10,PONIENTE,"{""type"": ""Polygon"", ""coordinates"": [[[-99.2137...","19.3163141213,-99.216257215",ALVARO OBREGON,SAN ANGEL,3,0110,"MULTIPOLYGON (((-99.21377 19.32112, -99.21365 ..."
2,3,2,1,4,CENTRO,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1485...","19.4587306491,-99.1427404298",CUAUHTEMOC,TLATELOLCO,8,014,"MULTIPOLYGON (((-99.14859 19.45707, -99.14733 ..."
3,4,3,1,5,CENTRO,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1424...","19.462120565,-99.1394016618",CUAUHTEMOC,TLATELOLCO,8,015,"MULTIPOLYGON (((-99.14247 19.46449, -99.14151 ..."
4,5,4,1,8,CENTRO,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1296...","19.4544010993,-99.1268359933",CUAUHTEMOC,TLATELOLCO,8,018,"MULTIPOLYGON (((-99.12969 19.45434, -99.12806 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
842,843,842,4,3,ORIENTE,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1275...","19.257393593,-99.1160688878",XOCHIMILCO,LA NORIA,1,043,"MULTIPOLYGON (((-99.12756 19.26873, -99.12623 ..."
843,844,843,4,6,ORIENTE,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1413...","19.2384413088,-99.1418800567",XOCHIMILCO,LA NORIA,1,046,"MULTIPOLYGON (((-99.14139 19.25101, -99.14120 ..."
844,845,844,4,5,ORIENTE,"{""type"": ""Polygon"", ""coordinates"": [[[-99.1413...","19.2502400945,-99.1293319747",XOCHIMILCO,LA NORIA,1,045,"MULTIPOLYGON (((-99.14139 19.25101, -99.14033 ..."
845,846,845,3,8,SUR,"{""type"": ""Polygon"", ""coordinates"": [[[-99.2483...","19.3159535138,-99.2427241547",MAGDALENA CONTRERAS,SAN JERONIMO,2,038,"MULTIPOLYGON (((-99.24836 19.31368, -99.24835 ..."


In [ ]:
ca.retr

,colonia_cve,ROBO A CASA HABITACION SIN VIOLENCIA,VIVPAR_HAB,VPH_3YMASC,Concentración de Desventajas
0,32,0.0,878.0,1.045558,-2.401170
1,619,9.0,1491.0,0.998659,-1.493854
2,1792,2.0,775.0,0.752258,3.328881
3,1774,0.0,284.0,0.989437,-2.071739
4,583,13.0,2941.0,0.956477,-1.218791
...,...,...,...,...,...
1804,1820,9.0,1756.0,0.801822,-0.232755
1805,1731,26.0,6070.0,0.737068,1.642092
1806,1799,4.0,2680.0,0.766045,2.415283
1807,1800,7.0,2569.0,0.800311,1.667266


In [ ]:
manzanas = gpd.read_file("datos/descargas/manzanas_2020_cdmx.zip")
carpetas = get_carpetas_desde_archivo("datos/descargas/carpetas_fiscalia.csv")
carpetas = carpetas.to_crs(manzanas.crs)

In [ ]:
carpetas = (gpd
            .sjoin_nearest(carpetas, manzanas[['CVEGEO', 'geometry']], distance_col='distancia')
            .drop(columns='index_right'))
carpetas.head()

,ao_hechos,mes_hechos,fecha_hechos,ao_inicio,mes_inicio,fecha_inicio,delito,fiscalia,agencia,unidad_investigacion,...,calle_hechos2,colonia_hechos,alcaldia_hechos,competencia,longitud,latitud,tempo,geometry,CVEGEO,distancia
0,2016.0,Enero,2016-01-31 22:16:00,2016,Febrero,2016-02-01 00:25:44,DAÑO EN PROPIEDAD AJENA INTENCIONAL,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-2,UI-3CD,...,LAGO PTE.,AMÉRICAS UNIDAS,BENITO JUAREZ,NaN,-99.140215,19.380315,NaN,POINT (2799543.997 823272.928),0901400010505010,11.600893
402,2016.0,Febrero,2016-02-03 01:17:00,2016,Febrero,2016-02-03 15:46:07,ROBO A NEGOCIO SIN VIOLENCIA,"INVESTIGACIÓN PARA LA ATENCIÓN DE NIÑOS, NIÑAS...",57,UI-3CD,...,NaN,AMÉRICAS UNIDAS,BENITO JUAREZ,NaN,-99.140222,19.380262,NaN,POINT (2799543.381 823267.064),0901400010505010,11.764028
458,2016.0,Febrero,2016-02-03 19:00:00,2016,Febrero,2016-02-03 20:51:15,ROBO A NEGOCIO SIN VIOLENCIA,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-2,UI-3CD,...,NaN,AMÉRICAS UNIDAS,BENITO JUAREZ,NaN,-99.141166,19.380219,NaN,POINT (2799444.590 823260.358),0901400010505010,0.000000
1557,2016.0,Febrero,2016-02-09 22:00:00,2016,Febrero,2016-02-09 23:50:19,ROBO A NEGOCIO SIN VIOLENCIA,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-3CD,...,NaN,AMÉRICAS UNIDAS,BENITO JUAREZ,NaN,-99.141166,19.380219,NaN,POINT (2799444.590 823260.358),0901400010505010,0.000000
1721,2016.0,Enero,2016-01-25 12:30:00,2016,Febrero,2016-02-10 21:01:00,DENUNCIA DE HECHOS,INVESTIGACIÓN EN BENITO JUÁREZ,BJ-1,UI-1CD,...,NaN,AMÉRICAS UNIDAS,BENITO JUAREZ,NaN,-99.142045,19.379627,NaN,POINT (2799353.855 823193.180),0901400010505010,0.348119


In [ ]:
carpetas.distancia.describe()

count    1.344566e+06
mean     4.424332e+00
std      8.087393e+01
min      0.000000e+00
25%      8.423795e-02
50%      2.879574e+00
75%      5.837401e+00
max      9.282383e+04
Name: distancia, dtype: float64

In [ ]:
carpetas.sort_values('distancia')

,ao_hechos,mes_hechos,fecha_hechos,ao_inicio,mes_inicio,fecha_inicio,delito,fiscalia,agencia,unidad_investigacion,...,calle_hechos2,colonia_hechos,alcaldia_hechos,competencia,longitud,latitud,tempo,geometry,CVEGEO,distancia
1257114,2021.0,Julio,2021-07-12 13:00:00,2021,Julio,2021-07-14 11:03:34,FRAUDE,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN IZTAP...,UAT-IZP-5,5 SIN DETENIDO,...,AVENIDA JALISCO,LEYES DE REFORMA 2A SECCIÓN,IZTAPALAPA,FUERO COMUN,-99.069921,19.365441,NaN,POINT (2806939.359 821780.092),0900700010572031,0.000000
1335383,2021.0,Noviembre,2021-11-13 14:30:00,2021,Noviembre,2021-11-13 15:23:07,LESIONES CULPOSAS POR TRANSITO VEHICULAR EN CO...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN TLALPAN,TLP-3,UI-1CD,...,NaN,SAN PEDRO MÁRTIR,TLALPAN,FUERO COMUN,-99.171202,19.266215,NaN,POINT (2796547.126 810615.720),0901200012009024,0.000000
1375303,2022.0,Enero,2022-01-18 09:20:00,2022,Enero,2022-01-18 12:41:00,ABUSO SEXUAL,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN TLALPAN,UAT-TLP-3,UI-1SD,...,.,SAN PEDRO MÁRTIR,TLALPAN,FUERO COMUN,-99.171701,19.265116,NaN,POINT (2796497.272 810493.383),0901200012009024,0.000000
560281,2019.0,Junio,2019-06-07 19:40:00,2019,Junio,2019-06-08 15:14:52,ROBO A PASAJERO A BORDO DE METRO SIN VIOLENCIA,INVESTIGACIÓN EN AGENCIAS DE ATENCIÓN ESPECIAL...,STCMP,UI-3CD,...,NaN,CENTRO,CUAUHTEMOC,NaN,-99.132840,19.425670,NaN,POINT (2800216.304 828293.531),0901500010979012,0.000000
556982,2019.0,Julio,2019-07-07 13:30:00,2019,Julio,2019-07-07 14:28:12,ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA,INVESTIGACIÓN EN CUAUHTEMOC,CUH-2,UI-2CD,...,JOSE MARIA PINO SUAREZ,CENTRO,CUAUHTEMOC,NaN,-99.132720,19.425220,NaN,POINT (2800229.861 828244.123),0901500010979012,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974052,2020.0,Febrero,2020-02-03 11:40:00,2020,Marzo,2020-03-04 09:05:06,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,FISCALÍA DE INVESTIGACIÓN TERRITORIAL EN GUSTA...,GAM-5,UI-3CD,...,AVENIDA 5,SAN JUAN DE ARAGÓN IV SECCIÓN,GUSTAVO A MADERO,NaN,-99.050386,19.483464,NaN,POINT (2808717.603 834846.107),0900500011542023,1381.122865
1074007,2020.0,Agosto,2020-08-31 10:30:00,2020,Septiembre,2020-09-18 12:42:10,ROBO DE OBJETOS DEL INTERIOR DE UN VEHICULO,AGENCIA DE DENUNCIA DIGITAL,CEN-1,Unidad de Investigación 1 S/D,...,NaN,NaN,LA MAGDALENA CONTRERAS,FUERO COMUN,-99.279597,19.215153,NaN,POINT (2785292.212 804759.800),0901201351227025,4468.728462
184618,2016.0,Abril,2016-04-13 12:45:00,2016,Abril,2016-04-13 22:16:06,ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...,INVESTIGACIÓN EN TLALPAN,TLP-3,UI-1CD,...,"PASANDO EL ARCO DE ""BUEN VIAJE LE DESEA LA DEL...",SAN MIGUEL TOPILEJO,TLALPAN,NaN,-99.209526,19.095354,NaN,POINT (2792897.103 791676.519),0901200192475012,4573.347452
395475,2016.0,Diciembre,2016-12-16 15:00:00,2017,Enero,2017-01-11 10:18:03,DENUNCIA DE HECHOS,INVESTIGACIÓN PARA LA ATENCIÓN DE DELITOS SEXU...,FDS-6,FDS-6-01,...,NaN,SANTO TOMAS AJUSCO,TLALPAN,NaN,-99.276546,19.206777,NaN,POINT (2785629.745 803841.291),0901201351227025,4756.899945


In [ ]:
carpetas.distancia.mean()

4.424332378725047